In [ ]:


from IPython import display
from collections import deque

# import socket
# import threading
from multiprocessing import Manager, Process, Queue

import time
from lib_xml_tree import *
from lib_connection import *
from lib_config_loader import load_config
from lib_manager_buffer import *


def routine_for_camera(server_address, port, mess_input, mess_output):
    conn = create_socket(server_address, port)
    
    # init variables supposed to send to camera
    watchDog_out = '0' 
    frame_assigned = '0' 
    position_reached = '0' 
    need_cam_cal = '0'        
    screenshot = '0' 
    
    
    while True:
        telegram_from_robot = None
        try:
            # get message from robot (dict)
            # if len(mess_input) > 0:
            # if mess_input.has_message():
            ################################################
            #RECEIVING MESSAGES
            ################################################ 
            
            # GET MESSAGE FROM ROBOT SERVER-SUBPROCESS
            telegram_from_robot = mess_input.get()
            if telegram_from_robot is not None:
                # extract data from telegram
                try:
                    watchDog_out = telegram_from_robot["WatchDog_out"]
                    frame_assigned = telegram_from_robot["Frame_assigned"]
                    position_reached = telegram_from_robot["Position_reached"]
                    need_cam_cal = telegram_from_robot["Need_cam_cal"]
                    screenshot = telegram_from_robot["Screenshot"]
                    ipoc = telegram_from_robot["IPOC"]
                    
                except Exception as ex:
                    print(f" Data can not be extracted from telegram from robot: {ex}")                
            # else:
            #     print('camera has no messages from robot')

            # GET MESSAGE FROM CAMERA CLIENT  
            bytesAddressPair = conn.recvfrom(4096)            
            ReceivedMessage = bytesAddressPair[0]
            received_address = bytesAddressPair[1]
            # # clientMsg = "Message from { } : {}".format(received_address, ReceivedMessage)
            # clientIP = "Client IP Address:{}".format(received_address)
            # # print(clientMsg)
            # # print(clientIP)    
            received_dict = extract_xml(ReceivedMessage)



            ################################################
            #SENDING MESSAGES
            ################################################           

            # SEND MESSAGE TO ROBOT SERVER-SUBPROCESS
            mess_output.put(received_dict)
            
                     
            # SEND MESSAGE TO CAMERA CLIENT
            sent_mess_list = list() # This message to be sent from Server to Camera client
            sent_mess_list.append({"Sen" : {'Type' : 'ServerToCamera'}})    
            sent_mess_list.append({"WatchDog_out" : watchDog_out})  
            sent_mess_list.append({'Frame_assigned': frame_assigned})
            sent_mess_list.append({'Position_reached': position_reached})
            sent_mess_list.append({'Need_cam_cal': need_cam_cal})
            sent_mess_list.append({'Screenshot': screenshot})            
            #creating a new telegram to send
            # telegram = create_xml(sent_mess_list)    
            telegram = create_xml_fast(sent_mess_list)
            # print(telegram)    
            SendData(telegram, conn, received_address)
            

            time.sleep(0.001)

        except Exception as e:
            print(f"[Server] Error in client thread: {e}")
            break



def routine_for_robot(server_address, port, mess_input, mess_output):
    conn = create_socket(server_address, port)
    
    # init variables supposed to send to robot
    watchDog_in = '0'
    frame_assign = '0'
    move_next_pt = '0'
    cam_cal_in_proc = '0'
    see_4_targets = '0'
    xyz1 = {'X': '0.0', 'Y': '0.0', 'Z': '0.0'}
    xyz2 = {'X': '0.0', 'Y': '0.0', 'Z': '0.0'}
    xyz3 = {'X': '0.0', 'Y': '0.0', 'Z': '0.0'}
    xyz4 = {'X': '0.0', 'Y': '0.0', 'Z': '0.0'}
    cam_cal_res = {'X': '0.0', 'Y': '0.0', 'Z': '0.0', 'A': '0.0', 'B': '0.0', 'C': '0.0'}
    
    while True:
        telegram_from_camera = None
        try:
            # get message from camera (dict)
            # if len(mess_input) > 0:
            # if mess_input.has_message(): 
            ################################################
            #RECEIVING MESSAGES
            ################################################           

            # GET MESSAGE FROM CAMERA SERVER-SUBPROCESS
            telegram_from_camera = mess_input.get()
            if telegram_from_camera is not None:
            # extract data from telegram
                try:
                    watchDog_in = telegram_from_camera["WatchDog_in"]
                    frame_assign = telegram_from_camera["Frame_assign"]
                    move_next_pt = telegram_from_camera["Move_next_pt"]
                    cam_cal_in_proc = telegram_from_camera["Cam_cal_in_proc"]
                    see_4_targets = telegram_from_camera["See_4_targets"]
                    xyz1 = telegram_from_camera["XYZ1"]
                    xyz2 = telegram_from_camera["XYZ2"]
                    xyz3 = telegram_from_camera["XYZ3"]
                    xyz4 = telegram_from_camera["XYZ4"]
                    cam_cal_res = telegram_from_camera["CAM_CAL_RES"]
                    
                    
                except Exception as ex:
                    print(f" Data can not be extracted from telegram from camera: {ex}")
            # else:
            #     print('robot has no messages from camera')
                

            # GET MESSAGE FROM ROBOT CLIENT
            bytesAddressPair = conn.recvfrom(4096)            
            ReceivedMessage = bytesAddressPair[0]
            received_address = bytesAddressPair[1]
            # clientMsg = "Message from { } : {}".format(received_address, ReceivedMessage)
            # clientIP = "Client IP Address:{}".format(received_address)
            # print('received from robot', ReceivedMessage)
            # print(clientIP)    
            received_dict = extract_xml(ReceivedMessage)


            

            ################################################
            #SENDING MESSAGES
            ################################################
            
            # SEND MESSAGE TO CAMERA SERVER-SUBPROCESS
            mess_output.put(received_dict)            
            
            # print(clientIP, received_dict)
            
            # SEND MESSAGE TO ROBOT CLIENT
            sent_mess_list = list() # This message to be sent from Server to KRC client
            sent_mess_list.append({"Sen" : {'Type' : 'ImFree'}})   
            sent_mess_list.append({'XYZ1': xyz1})
            sent_mess_list.append({'XYZ2': xyz2})
            sent_mess_list.append({'XYZ3': xyz3})
            sent_mess_list.append({'XYZ4': xyz4})
            sent_mess_list.append({'CAM_CAL_RES': cam_cal_res})
            sent_mess_list.append({"WatchDog_in" : watchDog_in}) 
            sent_mess_list.append({"Frame_assign" : frame_assign}) 
            sent_mess_list.append({"Move_next_pt" : move_next_pt}) 
            sent_mess_list.append({"Cam_cal_in_proc" : cam_cal_in_proc}) 
            sent_mess_list.append({"See_4_targets" : see_4_targets}) 
            # sent_mess_list.append({'RKorr': {'X': '278.60', 'Y': '164.79', 'Z': '615.22', 'A': '0.0', 'B': '0.0', 'C': '0.0'}})  
            sent_mess_list.append({'IPOC': received_dict['IPOC']})
            
            #creating a new telegram to send
            # telegram = create_xml(sent_mess_list)
            telegram = create_xml_fast(sent_mess_list)
            # print('to send to robot', telegram)    
            SendData(telegram, conn, received_address)

            time.sleep(0.001)

        except Exception as e:
            print(f"[Server] Error in client thread: {e}")
            break


if __name__ == '__main__':
    
    # Load IP, PORT configuration from config
    config = load_config()
    server_address = config['server']['address']
    robot_port = config['server']['robot_port']
    camera_port = config['server']['camera_port']

    

    # Bind server's address and port to expect message from client
    # conn_to_client_robot = create_socket(address = server_address, port = robot_port)
    # conn_to_client_camera = create_socket(address = server_address, port = camera_port)

    # Create shared messages from robot to camera and vice versa
    with Manager() as manager:
        mess_rob_to_cam = SingleMessageQueue(manager)
        mess_cam_to_rob = SingleMessageQueue(manager)


        p1 = Process(target=routine_for_robot, args=(server_address, robot_port, mess_cam_to_rob, mess_rob_to_cam))
        p2 = Process(target=routine_for_camera, args=(server_address, camera_port, mess_rob_to_cam, mess_cam_to_rob))

        p1.start()
        p2.start()
    
        print('UDP server is turning on (multiprocessing mode)')
        try:
            while True:
                time.sleep(0.001)
                display.clear_output(wait=True)
        except KeyboardInterrupt:
            print("Shutting down...")
    
        # Остановка процессов
        p1.terminate()
        p2.terminate()
        p1.join()
        p2.join()


                
        

In [ ]:
# Creates structure for ambedding into the xml package, which will be send to robot
def create_Rkorr_for_sending(frame = {'X' : 0, 'Y' : 0, 'Z' : 0}):  #frame = {'X' : float, 'Y' : float, 'Z' : float, 'A' : float, 'B' : float, 'C' : float}
    # print(frame)
    Rkorr = {'X' : str(frame['X']), 
             'Y' : str(frame['Y']), 
             'Z' : str(frame['Z'])}
    
    return Rkorr


def create_xml(target_frame_1 = {'X' : 0.100, 'Y' : 1.000, 'Z' : 0.030, 'A' : 0.000, 'B' : 0.000, 'C' : 0.000},
              target_frame_2 = {'X' : 0, 'Y' : 0, 'Z' : 0},
              target_frame_3 = {'X' : 0, 'Y' : 0, 'Z' : 0},
              target_frame_4 = {'X' : 0, 'Y' : 0, 'Z' : 0},
              IPOCtimerReseived = '123456789'):

    SenAttr = dict(Type='ImFree')

    root = xml.Element("Sen", SenAttr)
    
    ################# RESTORE IT ##################
    RkorrArrt_1 = create_Rkorr_for_sending(target_frame_1)
    Rkorr_1 = xml.SubElement(root, "RKorr", RkorrArrt_1)
    ################################################
    #print(RkorrArrt_1)
    #print(Rkorr_1)





    #RkorrArrt = dict(X='0.03')
    #Rkorr = xml.SubElement(root, "RKorr", RkorrArrt)
    #print(RkorrArrt) 
   




    
#     RkorrArrt_2 = create_Rkorr_for_sending(target_frame_2)
#     Rkorr_2 = xml.SubElement(root, "Rkorr", RkorrArrt_2)
    
#     RkorrArrt_3 = create_Rkorr_for_sending(target_frame_3)
#     Rkorr_3 = xml.SubElement(root, "Rkorr", RkorrArrt_3)
    
#     RkorrArrt_4 = create_Rkorr_for_sending(target_frame_4)
#     Rkorr_4 = xml.SubElement(root, "Rkorr", RkorrArrt_4)
    
    
    IPOC = xml.SubElement(root, "IPOC")
    IPOC.text = IPOCtimerReseived
    # IPOC.text = '123456789'
    
    
    tree = xml.ElementTree(root)
    
    
    DecodedTree = xml.tostring(root, encoding='utf8', method='xml', short_empty_elements='true', xml_declaration=False)
    return(DecodedTree)



In [3]:
create_xml(target_frame_1 = {'X' : 1, 'Y' : 2, 'Z' : 3})

b'<Sen Type="ImFree"><RKorr X="1" Y="2" Z="3" /><IPOC>123456789</IPOC></Sen>'

In [2]:
# Creates structure for ambedding into the xml package, which will be send to robot
def create_Rkorr_for_sending(frame = {'X' : 0, 'Y' : 0, 'Z' : 0, 'A' : 0, 'B' : 0, 'C' : 0}):  #frame = {'X' : float, 'Y' : float, 'Z' : float, 'A' : float, 'B' : float, 'C' : float}
    # print(frame)
    Rkorr = dict(X = str(frame['X']), 
             Y = str(frame['Y']), 
             Z = str(frame['Z']),
             A = str(frame['A']), 
             B = str(frame['B']), 
             C = str(frame['C'])
			)
    
    return Rkorr

def create_xml(target_frame_1 = {'X' : 0.100, 'Y' : 1.000, 'Z' : 0.030, 'A' : 0.000, 'B' : 0.000, 'C' : 0.000},
              target_frame_2 = {'X' : 0, 'Y' : 0, 'Z' : 0},
              target_frame_3 = {'X' : 0, 'Y' : 0, 'Z' : 0},
              target_frame_4 = {'X' : 0, 'Y' : 0, 'Z' : 0},
              IPOCtimerReseived = '123456789'):

    SenAttr = dict(Type='ImFree')

    root = xml.Element("Sen", SenAttr)
    
    ################# RESTORE IT ##################
    RkorrArrt_1 = create_Rkorr_for_sending(target_frame_1)
    Rkorr_1 = xml.SubElement(root, "RKorr", RkorrArrt_1)
    ################################################
    #print(RkorrArrt_1)
    #print(Rkorr_1)





    #RkorrArrt = dict(X='0.03')
    #Rkorr = xml.SubElement(root, "RKorr", RkorrArrt)
    #print(RkorrArrt) 
   




    
#     RkorrArrt_2 = create_Rkorr_for_sending(target_frame_2)
#     Rkorr_2 = xml.SubElement(root, "Rkorr", RkorrArrt_2)
    
#     RkorrArrt_3 = create_Rkorr_for_sending(target_frame_3)
#     Rkorr_3 = xml.SubElement(root, "Rkorr", RkorrArrt_3)
    
#     RkorrArrt_4 = create_Rkorr_for_sending(target_frame_4)
#     Rkorr_4 = xml.SubElement(root, "Rkorr", RkorrArrt_4)
    
    
    IPOC = xml.SubElement(root, "IPOC")
    IPOC.text = IPOCtimerReseived
    # IPOC.text = '123456789'
    
    
    tree = xml.ElementTree(root)
    
    
    DecodedTree = xml.tostring(root, encoding='utf8', method='xml', short_empty_elements='true', xml_declaration=False)
    return(DecodedTree)

In [3]:
xml_parcel = create_xml()
xml_parcel

b'<Sen Type="ImFree"><RKorr X="0.1" Y="1.0" Z="0.03" A="0.0" B="0.0" C="0.0" /><IPOC>123456789</IPOC></Sen>'

In [16]:
def create_Rkorr_for_sending(frame = {'X' : 0, 'Y' : 0, 'Z' : 0, 'A' : 0, 'B' : 0, 'C' : 0}):  #frame = {'X' : float, 'Y' : float, 'Z' : float, 'A' : float, 'B' : float, 'C' : float}
    # print(frame)
    Rkorr = dict(X = str(frame['X']), 
             Y = str(frame['Y']), 
             Z = str(frame['Z'])
			)
    
    return Rkorr

def create_xml(target_frame_1 = {'X' : 0.1, 'Y' : 0.2, 'Z' : 0.3},
              target_frame_2 = {'X' : 0, 'Y' : 0, 'Z' : 0},
              target_frame_3 = {'X' : 0, 'Y' : 0, 'Z' : 0},
              target_frame_4 = {'X' : 0, 'Y' : 0, 'Z' : 0},
              IPOCtimerReseived = '123456789'):

    SenAttr = dict(Type='ImFree')

    root = xml.Element("Sen", SenAttr)

    target_dict_1 = create_Rkorr_for_sending(target_frame_1)
    target_sub_el_1 = xml.SubElement(root, "XYZ1", target_dict_1)

    target_dict_2 = create_Rkorr_for_sending(target_frame_2)
    target_sub_el_2 = xml.SubElement(root, "XYZ2", target_dict_2)

    target_dict_3 = create_Rkorr_for_sending(target_frame_3)
    target_sub_el_3 = xml.SubElement(root, "XYZ3", target_dict_3)

    target_dict_4 = create_Rkorr_for_sending(target_frame_4)
    target_sub_el_4 = xml.SubElement(root, "XYZ4", target_dict_4)

    D_input_1 = xml.SubElement(root, "IPOC")
    D_input_1.text = str(1)




    
    ################# RESTORE IT ##################
#     RkorrArrt_1 = create_Rkorr_for_sending(target_frame_1)
#     Rkorr_1 = xml.SubElement(root, "RKorr", RkorrArrt_1)
    ################################################
#     print(RkorrArrt_1)
#     print(Rkorr_1)
    
#     RkorrArrt_2 = create_Rkorr_for_sending(target_frame_2)
#     Rkorr_2 = xml.SubElement(root, "Rkorr", RkorrArrt_2)
    
#     RkorrArrt_3 = create_Rkorr_for_sending(target_frame_3)
#     Rkorr_3 = xml.SubElement(root, "Rkorr", RkorrArrt_3)
    
#     RkorrArrt_4 = create_Rkorr_for_sending(target_frame_4)
#     Rkorr_4 = xml.SubElement(root, "Rkorr", RkorrArrt_4)
    
    
    IPOC = xml.SubElement(root, "IPOC")
    IPOC.text = IPOCtimerReseived
    
    
    
    tree = xml.ElementTree(root)
    
    
    DecodedTree = xml.tostring(root, encoding='utf8', method='xml', short_empty_elements='true', xml_declaration=False)
    return(DecodedTree)

In [17]:
create_xml()

b'<Sen Type="ImFree"><XYZ1 X="0.1" Y="0.2" Z="0.3" /><XYZ2 X="0" Y="0" Z="0" /><XYZ3 X="0" Y="0" Z="0" /><XYZ4 X="0" Y="0" Z="0" /><IPOC>1</IPOC><IPOC>123456789</IPOC></Sen>'

In [ ]:
#calculated
X = -633.939
Y = -1455.747
Z = 1219.194

#$POS_ACT tool = 0, base = 0
X = -613.631
Y = -1448.92
Z = 1201.029
A = 168.276
B = 68.202
C = -154.673

#camera
X = 10
Y = 20
Z = 30
A = 0
B = 0
C = 0

#camera see object
X = 0
Y = 0
Z = 13
A = 0
B = 0
C = 0

In [49]:
xml_parcel

b'<Sen Type="ImFree"><RKorr X="0.1" Y="1.0" Z="0.03" A="0.0" B="0.0" C="0.0" /><IPOC>123456789</IPOC></Sen>'

In [2]:
def extract_xml(bytes_parcel, find = 'all'): # 'find' could be also the list of key-words
    res_dict = dict()
       
    try: # checkking if we can parse input data to ElementTree
        tree = xml.parse(io.BytesIO(bytes_parcel)) # xml.etree.ElementTree.ElementTree

    except TypeError:
        print('incorrect type of input data')

    else:
        try: # checking 'find'
            find = find.lower()
            

        except AttributeError: #operating with list
            root = tree.getroot()
            
            #print('cant make lowercase')
            for i_tag in find:
                elem = tree.find(i_tag)                
                try: # if incorrect key_word in list
                    if elem.text is not None:
                        res_dict[elem.tag] = elem.text    # if element contains text
                    else:
                        res_dict[elem.tag] = elem.attrib  # if element contains dict
                except AttributeError:
                    print('Data do not contain "{}" key-word'.format(i_tag))



        else: #checking ALL
            if find == 'all':
                root = tree.getroot()
                if root.text is not None:        
                    res_dict[root.tag] = root.text    # if element contains text
                else:
                    res_dict[root.tag] = root.attrib  # if element contains dict

                for elem in root:
                    if elem.text is not None:        
                        res_dict[elem.tag] = elem.text    # if element contains text
                    else:
                        res_dict[elem.tag] = elem.attrib  # if element contains dict
            else:
                raise AttributeError('Use "all" or the list of keywords')

            

    return res_dict


In [31]:
xml_parcel                               # bytes

tree = xml.parse(io.BytesIO(xml_parcel)) # xml.etree.ElementTree.ElementTree
root = tree.getroot()                    # xml.etree.ElementTree.Element

# sub_elems = list()
sub_elems = [elem for elem in root]
elem_IPOC = tree.find('IPOC')
elem_RKorr = tree.find('RKorr')
# print(elem_RKorr)
# elem_RKorr.tag
print(elem_RKorr.text)
# xml_parcel.getroot()

None


In [32]:
type(sub_elems[0])

xml.etree.ElementTree.Element

In [8]:
res_dict = extract_xml(telegram, find = ['XYZ1'])
res_dict

{'XYZ1': {'X': '0.1',
  'Y': '1.0',
  'Z': '0.01',
  'A': '0.0',
  'B': '0.0',
  'C': '0.0'}}

In [41]:
res_dict.pop('RKorr')

{'X': '0.1', 'Y': '1.0', 'Z': '0.03', 'A': '0.0', 'B': '0.0', 'C': '0.0'}

AttributeError: 'str' object has no attribute 'lowcase'

In [58]:
d = {'XYZ1': {'X': '0.1', 'Y': '1.0', 'Z': '0.01', 'A': '0.0', 'B': '0.0', 'C': '0.0'}}
s = 'stt'
type(s) == str

True

In [48]:
list(d.keys())

['XYZ1']

In [3]:
def create_xml(sent_mess_list):
    
    for num, elem in enumerate(sent_mess_list):
        key = list(elem.keys())[0]
        value = elem[key]
        if type(value) == dict: 
            if num == 0: 
                root = xml.Element(key , elem[key])
            else:
                element_tree = xml.SubElement(root, key , elem[key])
                
        elif type(value) == str:
            if num == 0:
                root = xml.Element(key)
                root.text = value 
            else:
                element_tree = xml.SubElement(root, key)
                element_tree.text = value                 
            
    tree = xml.ElementTree(root)    
    DecodedTree = xml.tostring(root, encoding='utf8', method='xml', short_empty_elements='true', xml_declaration=False)
    
    return(DecodedTree)

In [9]:
sent_mess_list = list() # List of dictionaries. every dictionary will be the element of ElementTree


sent_mess_list.append({'Sen' : {'Type' : 'ImFree'}})
# # # LIST OF 4 TARGETS ############################
# sent_mess_list.append({'XYZ1': {'X': '0.1', 'Y': '1.0', 'Z': '0.01', 'A': '0.0', 'B': '0.0', 'C': '0.0'}})
# # sent_mess_list.append({'XYZ2': {'X': '0.2', 'Y': '2.0', 'Z': '0.02', 'A': '0.0', 'B': '0.0', 'C': '0.0'}})
# # sent_mess_list.append({'XYZ3': {'X': '0.3', 'Y': '3.0', 'Z': '0.03', 'A': '0.0', 'B': '0.0', 'C': '0.0'}})
# # sent_mess_list.append({'XYZ4': {'X': '0.4', 'Y': '4.0', 'Z': '0.04', 'A': '0.0', 'B': '0.0', 'C': '0.0'}})  
sent_mess_list.append({'IPOC': '123'})
 
telegram = create_xml(sent_mess_list)
telegram

b'<Sen Type="ImFree"><IPOC>123</IPOC></Sen>'